# Assignment 4 - Predicting and understanding viewer engagement with educational videos


With the accelerating popularity of online educational experiences, the role of online lectures and other educational video continues to increase in scope and importance. Open access educational repositories such as videolectures.net, as well as Massive Open Online Courses (MOOCs) on platforms like Coursera, have made access to many thousands of lectures and tutorials an accessible option for millions of people around the world. Yet this impressive volume of content has also led to a challenge in how to find, filter, and match these videos with learners. This assignment gives you an example of how machine learning can be used to address part of that challenge.

# About the prediction problem

One critical property of a video is engagement: how interesting or "engaging" it is for viewers, so that they decide to keep watching. Engagement is critical for learning, whether the instruction is coming from a video or any other source. There are many ways to define engagement with video, but one common approach is to estimate it by measuring how much of the video a user watches. If the video is not interesting and does not engage a viewer, they will typically abandon it quickly, e.g. only watch 5 or 10% of the total.

A first step towards providing the best-matching educational content is to understand which features of educational material make it engaging for learners in general. This is where predictive modeling can be applied, via supervised machine learning. For this assignment, your task is to predict how engaging an educational video is likely to be for viewers, based on a set of features extracted from the video's transcript, audio track, hosting site, and other sources.

We chose this prediction problem for several reasons:

It combines a variety of features derived from a rich set of resources connected to the original data;
The manageable dataset size means the dataset and supervised models for it can be easily explored on a wide variety of computing platforms;
Predicting popularity or engagement for a media item, especially combined with understanding which features contribute to its success with viewers, is a fun problem but also a practical representative application of machine learning in a number of business and educational sectors.

# About the dataset

We extracted training and test datasets of educational video features from the VLE Dataset put together by researcher Sahan Bulathwela at University College London.

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single educational video, and includes information about diverse properties of the video content as described further below. The target variable is engagement which was defined as True if the median percentage of the video watched across all viewers was at least 30%, and False otherwise.

Note: Any extra variables that may be included in the training set are simply for your interest if you want an additional source of data for visualization, or to enable unsupervised and semi-supervised approaches. However, they are not included in the test set and thus cannot be used for prediction.

For this final assignment, you will bring together what you've learned across all four weeks of this course, by exploring different prediction models for this new dataset. In addition, we encourage you to apply what you've learned about model selection to do hyperparameter tuning using training/validation splits of the training data, to optimize the model and further increase its performance. In addition to a basic evaluation of model accuracy, we've also provided a utility function to visualize which features are most and least contributing to the overall model performance.

#### Data fields

title_word_count - the number of words in the title of the video.

document_entropy - a score indicating how varied the topics are covered in the video, based on the transcript. Videos with smaller entropy scores will tend to be more cohesive and more focused on a single topic.

freshness - The number of days elapsed between 01/01/1970 and the lecture published date. Videos that are more recent will have higher freshness values.

easiness - A text difficulty measure applied to the transcript. A lower score indicates more complex language used by the presenter.

fraction_stopword_presence - A stopword is a very common word like 'the' or 'and'. This feature computes the fraction of all words that are stopwords in the video lecture transcript.

speaker_speed - The average speaking rate in words per minute of the presenter in the video.

silent_period_rate - The fraction of time in the lecture video that is silence (no speaking).


#### train.csv only:

engagement - Target label for training. True if learners watched a substantial portion of the video (see description), or False otherwise.

# Evaluation

Your predictions will be given as the probability that the corresponding video will be engaging to learners.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model with an AUC (area under ROC curve) of at least 0.8 passes this assignment, and over 0.85 will receive full points. ___

For this assignment, create a function that trains a model to predict significant learner engagement with a video using asset/train.csv. Using this model, return a Pandas Series object of length 2309 with the data being the probability that each corresponding video from readonly/test.csv will be engaging (according to a model learned from the 'engagement' label in the training set), and the video index being in the id field.

# Import libraries

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [20]:
def engagement_model():

    # Load datasets
    train_path = "train.csv"
    test_path = "test.csv"
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # Prepare training data
    X = train_df.drop(columns=["id", "engagement"])  # Features
    y = train_df["engagement"].astype(int)  # Encode target as 0/1

    # Split into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Train RandomForestClassifier
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # Predict on validation set and calculate AUC
    y_val_probs = rf_model.predict_proba(X_val)[:, 1]  # Get probability of class 1 (engaging)
    auc_score = roc_auc_score(y_val, y_val_probs)
    print(f"Validation AUC Score: {auc_score:.4f}")

    # Prepare test data
    X_test = test_df.drop(columns=["id"])

    # Predict engagement probabilities
    test_probs = rf_model.predict_proba(X_test)[:, 1]

    # Format results as a Pandas Series with 'id' as the index
    submission = pd.Series(test_probs, index=test_df["id"], name="engagement")

    # Save predictions to CSV
    submission.to_csv("submission.csv", header=True)

    print("Predictions saved as submission.csv")
   
    print(submission)

In [24]:
stu_ans = engagement_model()
#assert isinstance(stu_ans, pd.Series), "Your function should return a pd.Series. "
#assert len(stu_ans) == 2309, "Your series is of incorrect length: expected 2309 "
#assert np.issubdtype(stu_ans.index.dtype, np.integer), "Your answer pd.Series should have an index of integer type representing video id."


Validation AUC Score: 0.8980
Predictions saved as submission.csv
id
9240     0.01
9241     0.04
9242     0.04
9243     0.90
9244     0.04
         ... 
11544    0.01
11545    0.00
11546    0.01
11547    0.94
11548    0.03
Name: engagement, Length: 2309, dtype: float64
